In [2]:
import sys
print(sys.executable)
!python --version

/storage/ice1/1/0/vchopra37/projects/edge_glass/edge_glass_env/bin/python
Python 3.12.5


In [10]:
import os

In [11]:
os.getcwd()

'/storage/ice1/1/0/vchopra37/projects/edge_glass'

In [5]:
# ! uv pip install torch torchvision torchaudio
# ! uv pip install transformers sentencepiece accelerate
# ! uv pip install soundfile librosa
# ! uv pip install pillow

In [6]:
# scripts/test_image_encoder.py
import torch
from PIL import Image
from pathlib import Path
from transformers import CLIPModel, CLIPProcessor

/storage/ice1/1/0/vchopra37/projects/edge_glass/edge_glass_env/lib/python3.12/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
def main():
    

    img_path = Path("data/sample1.jpg")
    image = Image.open(img_path).convert("RGB")

    texts = ["a dog", "a cat", "a man with a laptop"]

    # processor handles both image + text
    inputs = processor(text=texts, images=image, return_tensors="pt", padding=True).to(device)

    with torch.no_grad():
        out = model(**inputs)
        img_emb = out.image_embeds           # shape: (1, 512)
        text_embs = out.text_embeds          # shape: (len(texts), 512)

    print("Image embedding shape:", img_emb.shape)
    print("Text embeddings shape:", text_embs.shape)
    print("Image embedding (first 5 vals):", img_emb[0, :5])

    # cosine with normalization
    img_norm = img_emb / img_emb.norm(dim=-1, keepdim=True)
    txt_norm = text_embs / text_embs.norm(dim=-1, keepdim=True)
    sims = (img_norm @ txt_norm.T).squeeze(0)   # (len(texts),)

    for t, s in zip(texts, sims.tolist()):
        print(f"{t:30s} -> {s:.4f}")
